In [1]:
%reload_ext autoreload
%autoreload 2

In [36]:
import os
import pprint
import io

import pymongo
import json
import pandas as pd
from IPython import display

import climatools.lblnew.setup_overlap as setup_overlap
import climatools.lblnew.runrecord_overlap as runrecord_overlap
import climatools.lblnew.pipeline as pipeline

In [3]:
client = pymongo.MongoClient('localhost', 27017, connect=False)

In [4]:
db = client.lblnew

In [5]:
collection = db.overlap_lw

In [13]:
def make_doc(param=None):
    '''
    Make a Mongodb document for an lblnew-overlap run.
    
    Parameters
    ----------
    param: dict
        lblnew-overlap input parameters.
    doc: dict
        lblnew-overlap input (param) and output (flux and cooling rate)
    '''
    dir_fortran = pipeline.get_dir_case(param=param, 
                                        setup=setup_overlap)
    
    fnames = [n for n in os.listdir(dir_fortran) if n.endswith('.dat')]
    
    s_outputs = []
    for fname in fnames:
        with open(os.path.join(dir_fortran, fname),
                  mode='r', encoding='utf-8') as f:
            s = f.read()
            
        s_outputs.append(s)
        
    doc = {fname.split('.')[0]: s
           for fname, s in zip(fnames, s_outputs)}
    doc['param'] = param
    
    return doc    

In [14]:
params = runrecord_overlap.params()

In [20]:
docs = (make_doc(param=param) for param in params)
collection.insert_many(docs)

In [21]:
collection.count()

60

In [37]:
for r in collection.find({'param.molecule': {'h2o': 'atmpro',
                                     'co2': 400e-6}},
                ['param', 'output_wflux']):
    display.display(pd.read_csv(io.StringIO(r['output_wflux']), sep=r'\s+').head())
    print()

,igg,level,pressure,sfu,sfd,fnet
0,1,1,0.000000,-17.530033,0.000000,-17.530033
1,1,2,0.000624,-17.530397,0.003571,-17.526825
2,1,3,0.000876,-17.530547,0.004760,-17.525787
3,1,4,0.001229,-17.530743,0.006283,-17.524460
4,1,5,0.001723,-17.530997,0.008184,-17.522813


,igg,level,pressure,sfu,sfd,fnet
0,1,1,0.000000,-24.588660,0.000000,-24.588660
1,1,2,0.000624,-24.588676,0.000027,-24.588649
2,1,3,0.000876,-24.588683,0.000039,-24.588644
3,1,4,0.001229,-24.588692,0.000054,-24.588637
4,1,5,0.001723,-24.588704,0.000077,-24.588627


,igg,level,pressure,sfu,sfd,fnet
0,1,1,0.000000,-60.152945,0.000000e+00,-60.152945
1,1,2,0.000624,-60.152945,2.513004e-08,-60.152945
2,1,3,0.000876,-60.152945,3.542913e-08,-60.152945
3,1,4,0.001229,-60.152945,5.041285e-08,-60.152945
4,1,5,0.001723,-60.152945,7.226448e-08,-60.152945


,igg,level,pressure,sfu,sfd,fnet
0,1,1,0.000000,-4.022411,0.000000,-4.022411
1,1,2,0.000624,-4.022417,0.000027,-4.022390
2,1,3,0.000876,-4.022420,0.000033,-4.022387
3,1,4,0.001229,-4.022424,0.000039,-4.022385
4,1,5,0.001723,-4.022429,0.000045,-4.022384


,igg,level,pressure,sfu,sfd,fnet
0,1,1,0.000000,-14.100929,0.000000,-14.100929
1,1,2,0.000624,-14.102134,0.003694,-14.098440
2,1,3,0.000876,-14.102641,0.004969,-14.097672
3,1,4,0.001229,-14.103281,0.006725,-14.096556
4,1,5,0.001723,-14.104082,0.009084,-14.094999


...

,igg,level,pressure,sfu,sfd,fnet
0,1,1,0.000000,-1.146076,0.000000,-1.146076
1,1,2,0.000624,-1.146099,0.000030,-1.146069
2,1,3,0.000876,-1.146110,0.000037,-1.146073
3,1,4,0.001229,-1.146125,0.000048,-1.146077
4,1,5,0.001723,-1.146145,0.000063,-1.146082


,igg,level,pressure,sfu,sfd,fnet
0,1,1,0.000000,-15.774182,0.000000,-15.774182
1,1,2,0.000624,-15.774567,0.003598,-15.770969
2,1,3,0.000876,-15.774725,0.004797,-15.769928
3,1,4,0.001229,-15.774930,0.006337,-15.768593
4,1,5,0.001723,-15.775196,0.008262,-15.766933


,igg,level,pressure,sfu,sfd,fnet
0,1,1,0.000000,-25.641957,0.000000,-25.641957
1,1,2,0.000624,-25.641973,0.000028,-25.641946
2,1,3,0.000876,-25.641980,0.000039,-25.641941
3,1,4,0.001229,-25.641990,0.000055,-25.641934
4,1,5,0.001723,-25.642002,0.000078,-25.641924


,igg,level,pressure,sfu,sfd,fnet
0,1,1,0.000000,-65.019705,0.000000e+00,-65.019705
1,1,2,0.000624,-65.019705,2.556578e-08,-65.019705
2,1,3,0.000876,-65.019705,3.607621e-08,-65.019705
3,1,4,0.001229,-65.019705,5.143870e-08,-65.019705
4,1,5,0.001723,-65.019705,7.391145e-08,-65.019705


,igg,level,pressure,sfu,sfd,fnet
0,1,1,0.000000,-4.601203,0.000000,-4.601203
1,1,2,0.000624,-4.601210,0.000028,-4.601182
2,1,3,0.000876,-4.601213,0.000034,-4.601179
3,1,4,0.001229,-4.601217,0.000040,-4.601177
4,1,5,0.001723,-4.601223,0.000046,-4.601177
